In [4]:
from myparser import Parser
from random import choice
import string
# import nltk
import re
import csv
import sys

In [2]:
# "is.*(?=\s\bof\s)"
# relations_list = ["is a", "is the", "are", "is an", "was the", "were", "was a", "was an"]
# relation_patterns = [r"(?<=\b%s\s)((\w*)(?:\s*))*" % rel for rel in relations_list]
# relations_re = re.compile('|'.join(relation_patterns))

In [3]:
complete_pat = r" \b(and other|for a|(is|are|was|were|will be|to be)\s(the\s((kind(s)?|part(s)?|f" + \
r"orm(s)?|sort(s)?|type(s)?|set(s)?|range(s)?|body(s)?|style(s)?)\sof)?|an\s((kind" + \
r"(s)?|part(s)?|form(s)?|sort(s)?|type(s)?|set(s)?|range(s)?|body(s)?|style(s)?)\s" + \
r"of)?|a\s((kind(s)?|part(s)?|form(s)?|sort(s)?|type(s)?|set(s)?|range(s)?|body(s)" + \
r"?|style(s)?)\sof)?|some\s((kind(s)?|part(s)?|form(s)?|sort(s)?|type(s)?|set(s)?|" + \
r"range(s)?|body(s)?|style(s)?)\sof)?|one\s((kind(s)?|part(s)?|form(s)?|sort(s)?|t" + \
r"ype(s)?|set(s)?|range(s)?|body(s)?|style(s)?)\sof)?)?)\b\s?([ \w\_\-\&\s\"]+?)\s" + \
r"?\b(\.|\,|(or|and|by|that|around|of|on|from|at|in|with|who|where|what|whose|whic" + \
r"h|made|done|written|[a-z]*ed)\b)"

(is|mean)\s+((the|a|an)\s+)?(\w+\s+)*(?P<type>\w+)(\s+(from|in|of|that|for))
\b(is|mean)\b\s+(\b(the|a|an)\b\s+)?(\w+\s+)*(?P<type>\w+)(\s+\b(from|in|of|that|for)\b)
<_sre.SRE_Match object; span=(24, 50), match='is the political leader of'>
leader


In [49]:
sum(map(lambda val: val!="", types.values()))

46850

In [65]:
verbs = ["is", "are", "mean", "means", "was", "were", "will be", "to be"]
dets = ["the", "a", "an", "The"]
previous_noun = ["part(s)?", "type(s)?", "one(s)?", "style(s)?", "word(s)?", "set(s)?"]
end_wds = ["from", "in", "of", "that", "for", "with", "and", "around",
           "or", "on", "who", "where", "what", "which", "at", "by", "[a-z]+ed", "made"]

end_pat = r"\b(\.|\,|(" + r"|".join(end_wds) + r")\b)"
type_pat = r"(?P<type>[ \w\_\-\&\s\"]+?)\s?"

complete_pat = r"\b(" + r"|".join(verbs) + r")\s+(((" + r"|".join(dets) + r")\s+)?(\w+\s+)*(" + \
                r"|".join(previous_noun) + r")\s+(of|for)\s+)?((" + r"|".join(dets) + r")\s+)?" + type_pat + end_pat

global relations_re
relations_re = re.compile(complete_pat)
types = {}
for page in Parser('./wikipedia-first.txt'):
    types[page.title] = extractType(page)
scores = eval_acc_goldstd(types)
# print(debug_goldstd(types))
print("{0:.2%}".format(scores[0]))

84.09%


In [58]:
def extractType(page):

    matching = relations_re.search(page.content)
    if matching:
        typ = matching.group("type")
        return typ.split()[-1].strip()
    else:
        return ""

In [60]:
def eval_acc_goldstd(predicted_fact):

    with open('./gold-standard-sample.tsv', mode='r', encoding='utf-8') as infile:
        reader = csv.reader(infile, delimiter='\t')
        goldstd = {rows[0]:rows[1] for rows in reader}

    scores = list(map(lambda ent: goldstd[ent] == predicted_fact[ent],
                        goldstd.keys()))

    non_empty = list(map(lambda ent: predicted_fact[ent] != "",
                        goldstd.keys()))

    precision = float(sum(scores)) / sum(non_empty)
    rappel = float(sum(scores)) / len(scores)
    
    return precision, rappel

In [7]:
def debug_goldstd(predicted_fact):

    with open('./gold-standard-sample.tsv', mode='r', encoding='utf-8') as infile:
        reader = csv.reader(infile, delimiter='\t')
        goldstd = {rows[0]:rows[1] for rows in reader}

    scores = []
    with open('./results_on_goldstd.tsv', 'w', encoding="utf-8") as test:
        for ent in goldstd.keys():
            if goldstd[ent]==predicted_fact[ent]:
                scores.append(True)
            else:
                scores.append(False)
                test.write(ent + "\t" + goldstd[ent] + "\t" + predicted_fact[ent] + "\n")
    
    return float(sum(scores)) / len(scores)

In [18]:
verbs = ["is", "are", "mean", "means", "was", "were"]
dets = ["the", "a", "an"]
previous_noun = ["part", "type", "one", "style"]
end_wds = ["from", "in", "of", "that", "for", "with", "and", "around"]
complete_pat = []
# \w+ => \w+(\'|\-)?\w*
# r"(?P<type>\w+)" --> r"([\w\_\-\&\s\"]+?)\s?"
complete_pat.append(r"\b(" + r"|".join(verbs) + r")\b\s+(\b(" + r"|".join(dets) + \
                      r")\b\s+)?(\b("+ "|".join(previous_noun) + r")\b\s+\bof\b\s+)+(\b(" + r"|".join(dets) + \
                      r")\b\s+)?(\w+\s+)*?(?P<type>[\w\_\-\&\s\"]+?)((\s+\b\w+ed\b)?\s+\b(" + r"|".join(end_wds) + r")\b)(\s+\w+)*\.$")
complete_pat.append(r"\b(" + r"|".join(verbs) + r")\b\s+(\b(" + r"|".join(dets) + \
                          r")\b\s+)?(\w+\s+)*?(?P<type>[\w\_\-\&\s\"]+?)((\s+\b\w+ed\b)?\s+\b(" + r"|".join(end_wds) + r")\b)")
complete_pat.append(r"\b(" + r"|".join(verbs) + r")\b\s+(\b(" + r"|".join(dets) + r")\b\s+)?(\w+\s+)*(?P<type>[\w\_\-\&\s\"]+?)")
relations_re = []
for complete_re in complete_pat:
    relations_re.append(re.compile(complete_re))
types = {}
for page in Parser('./wikipedia-first.txt'):
    types[page.title] = extractType(page, relations_re)
# scores = eval_acc_goldstd(types)
print(debug_goldstd(types))

0.24444444444444444
